In [1]:
# standard libraries
import math
import os
import tempfile
import json
from pathlib import Path
import pickle

# standard scientific libraries
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from numpy import asarray, save, load
import pandas as pd
import seaborn as sns

# scikit-learn
import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

# tensorflow
import tensorflow as tf
import tensorflow_addons as tfa
import keras
from keras.models import Sequential
from keras.layers import Dense,Conv1D, Conv2D, MaxPooling2D, Dropout, Flatten, Input, MaxPooling1D
from keras.optimizers import RMSprop
from keras.utils import to_categorical

2024-08-01 17:32:46.689835: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/mimer/NOBACKUP/groups/naiss2024-5-16/tloop/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/mimer/NOBACKUP/groups/naiss2024-5-16/tloop/lib/python3.10/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons sup

# Model

In [2]:
def make_model(metrics, num_classes, input_shape, output_bias=None, pool_size = 2):
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)
    
    model = Sequential([        
        # First convolution
        Conv1D(16, 3, strides=1, activation='relu', padding='same', # TODO set as 64?
               input_shape = input_shape,
               kernel_initializer = 'he_normal',
               bias_initializer = 'zeros'),
        MaxPooling1D(pool_size = pool_size, strides = 2), # TODO set strides?
        Dropout(0.2),
        Conv1D(32, 3, strides = 1, activation = 'relu', padding = 'same',
               kernel_initializer = 'he_normal',
               bias_initializer = 'zeros'),
        MaxPooling1D(pool_size = pool_size, strides = 2),
        Dropout(0.2),
        Flatten(),
        
        # Neuron hidden layer
        Dense(int(input_shape[0]/pool_size) * 32, activation = 'relu', kernel_initializer='he_normal', bias_initializer = output_bias),
        Dropout(0.2),
        
        # Output neuron
        # Dense(1, activation='sigmoid')  # Sigmoid for binary question. It will contain a value from 0-1 where 0 for class ('not GNRA') and 1 for the other ('GNRA')
        Dense(num_classes, activation='softmax', bias_initializer=output_bias, kernel_initializer='glorot_uniform') # TODO categorical: Softmax for multiclass classification
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate = 1e-3),  # optimizer=RMSprop(lr=0.001),
        loss=keras.losses.CategoricalCrossentropy(), 
        # loss=keras.losses.BinaryCrossentropy(), # TODO remove?
        metrics=metrics)

    return model

In [3]:
def train(DATA_DIR, NUM_CLASSES, EPOCHS = 20, BATCH_SIZE = 16):
    DATA_DIR = Path(DATA_DIR)
    SET_NAME = str(DATA_DIR).split("/")[-1]
    RESULTS_FILE = RESULTS_DIR / SET_NAME
    # RESULTS_FILE = Path(f"results/ANN/homology_reduced/{SET_NAME}")
    METRICS = [ 
        keras.metrics.TruePositives(name = 'tp'), # TODO is this correct? something is wrong
        keras.metrics.FalsePositives(name = 'fp'),
        keras.metrics.TrueNegatives(name = 'tn'),
        keras.metrics.FalseNegatives(name = 'fn'),
        # keras.metrics.BinaryAccuracy(name = 'accuracy'), # TODO remove this?
        keras.metrics.CategoricalAccuracy(name='accuracy'),
        keras.metrics.Precision(name = 'precision'),
        keras.metrics.Recall(name = 'recall'),
        keras.metrics.AUC(name = 'auc', curve='roc'),
        keras.metrics.AUC(name = 'prc', curve = 'PR'), # precision-recall curve
        tfa.metrics.F1Score(name = 'f1', num_classes = NUM_CLASSES),
        tfa.metrics.MatthewsCorrelationCoefficient(name = 'mcc', num_classes = NUM_CLASSES)
    ]

    # Load dataset
    
    train_dict = np.load(DATA_DIR/"train_matrices.npz", allow_pickle=True)
    x_train = np.stack(train_dict['arr_0'], axis=0)
    y_train = np.load(DATA_DIR/"train_labels.npy", allow_pickle=True)
    y_train = to_categorical(y_train, num_classes=NUM_CLASSES)

    dev_dict = np.load(DATA_DIR/"dev_matrices.npz", allow_pickle=True)
    x_dev = np.stack(dev_dict['arr_0'], axis=0)
    y_dev = np.load(DATA_DIR/"dev_labels.npy", allow_pickle=True)
    y_dev = to_categorical(y_dev, num_classes=NUM_CLASSES)

    print("Training features shape:", x_train.shape)
    print("Training labels shape:", y_train.shape)

    print("\nDev (validation) features shape:", x_dev.shape)
    print("Dev (validation) labels shape:", y_dev.shape)

    print("\nInput shape:", x_train.shape[1:])
    print()

    INPUT_SHAPE = x_train.shape[1:]

    # Calculate class weight

    # Scaling by total/2 helps keep the loss to a similar magnitude.
    # The sum of the weights of all examples stays the same.
    y_integers = np.argmax(y_train, axis=1)
    class_weights = np.round(class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_integers), y=y_integers), 2)
    d_class_weights = dict(enumerate(class_weights))

    # Train with class weights

    model = make_model(METRICS, NUM_CLASSES, INPUT_SHAPE)

    history = model.fit(
        x_train,
        y_train,
        validation_data = (x_dev, y_dev),
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        
        # The class weights go here
        class_weight=d_class_weights)
    
    # Save model
    model.save(RESULTS_FILE, overwrite=True)

    # Save history
    with open(RESULTS_FILE/"history", 'wb') as file:
        pickle.dump(history.history, file)

In [4]:
PATH = Path("../data_generation/training_data/homology_reduced/")
RESULTS_DIR = Path(f"results/ANN/homology_reduced/")
for dir in os.listdir(PATH):
    if dir[-1] != "U" and "gnravall" not in dir and os.path.isdir(PATH/dir):
        if "clusters" in dir:
            pass
            # print(dir)
            # train(PATH/dir, 24)
        else:
            # pass
            train(PATH/dir, 2)

2024-08-01 17:34:19.428716: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13762 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:87:00.0, compute capability: 7.5


Training features shape: (246, 8, 5)
Training labels shape: (246, 2)

Dev (validation) features shape: (31, 8, 5)
Dev (validation) labels shape: (31, 2)

Input shape: (8, 5)

Epoch 1/20
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


2024-08-01 17:34:23.020413: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401


16/16 [==============================] - 5s 68ms/step - loss: 0.7974 - tp: 128.0000 - fp: 118.0000 - tn: 128.0000 - fn: 118.0000 - accuracy: 0.5203 - precision: 0.5203 - recall: 0.5203 - auc: 0.5543 - prc: 0.5445 - f1: 0.5177 - mcc: 0.0372 - val_loss: 0.6187 - val_tp: 21.0000 - val_fp: 10.0000 - val_tn: 21.0000 - val_fn: 10.0000 - val_accuracy: 0.6774 - val_precision: 0.6774 - val_recall: 0.6774 - val_auc: 0.7201 - val_prc: 0.6995 - val_f1: 0.6477 - val_mcc: 0.3206
Epoch 2/20
16/16 [==============================] - 0s 8ms/step - loss: 0.7488 - tp: 145.0000 - fp: 101.0000 - tn: 145.0000 - fn: 101.0000 - accuracy: 0.5894 - precision: 0.5894 - recall: 0.5894 - auc: 0.5900 - prc: 0.5660 - f1: 0.5894 - mcc: 0.1798 - val_loss: 0.6148 - val_tp: 24.0000 - val_fp: 7.0000 - val_tn: 24.0000 - val_fn: 7.0000 - val_accuracy: 0.7742 - val_precision: 0.7742 - val_recall: 0.7742 - val_auc: 0.7435 - val_prc: 0.7146 - val_f1: 0.7704 - val_mcc: 0.5424
Epoch 3/20
16/16 [==============================] - 

INFO:tensorflow:Assets written to: results/ANN/homology_reduced/gnra_8/assets


INFO:tensorflow:Assets written to: results/ANN/homology_reduced/gnra_8/assets


Training features shape: (452, 24, 5)
Training labels shape: (452, 2)

Dev (validation) features shape: (57, 24, 5)
Dev (validation) labels shape: (57, 2)

Input shape: (24, 5)

Epoch 1/20
29/29 [==============================] - 3s 47ms/step - loss: 0.8415 - tp: 246.0000 - fp: 206.0000 - tn: 246.0000 - fn: 206.0000 - accuracy: 0.5442 - precision: 0.5442 - recall: 0.5442 - auc: 0.5603 - prc: 0.5420 - f1: 0.5410 - mcc: 0.0906 - val_loss: 0.5099 - val_tp: 44.0000 - val_fp: 13.0000 - val_tn: 44.0000 - val_fn: 13.0000 - val_accuracy: 0.7719 - val_precision: 0.7719 - val_recall: 0.7719 - val_auc: 0.8720 - val_prc: 0.8773 - val_f1: 0.7708 - val_mcc: 0.5654
Epoch 2/20
29/29 [==============================] - 0s 6ms/step - loss: 0.5635 - tp: 326.0000 - fp: 126.0000 - tn: 326.0000 - fn: 126.0000 - accuracy: 0.7212 - precision: 0.7212 - recall: 0.7212 - auc: 0.7820 - prc: 0.7704 - f1: 0.7166 - mcc: 0.4360 - val_loss: 0.4031 - val_tp: 48.0000 - val_fp: 9.0000 - val_tn: 48.0000 - val_fn: 9.0000 - 

INFO:tensorflow:Assets written to: results/ANN/homology_reduced/gnra_24/assets


INFO:tensorflow:Assets written to: results/ANN/homology_reduced/gnra_24/assets
